<a href="https://colab.research.google.com/github/estebanmartinez5/Kaggle_Saber_Pro_Colombia/blob/main/04_modelo_con_preprocesado_secundario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"danielloaizacarmona","key":"cf9d9029f8c19118e9f0c7abab4bac38"}'}

In [2]:
import os
import zipfile

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.12GB/s]


In [4]:
with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
    zip_ref.extractall("datos")


# **Carga y exploración de los datos**

In [5]:
import pandas as pd

data = pd.read_csv("datos/train.csv")

datos_unicos = len(set(data['ESTU_PRGM_ACADEMICO']))
print("Número de datos únicos en 'ESTU_PRGM_ACADEMICO':", datos_unicos)


educacion_madre_counts = dict(data['ESTU_PRGM_ACADEMICO'].value_counts().head())
print("Conteo de valores en 'ESTU_PRGM_ACADEMICO':\n", educacion_madre_counts)


conteo_valores = dict(data['ESTU_PRGM_ACADEMICO'].groupby(data['ESTU_PRGM_ACADEMICO']).size().head())
print("Agrupación por 'ESTU_PRGM_ACADEMICO':\n", conteo_valores)


column_names = [col for col in data.columns]
print("Nombres de las columnas:\n", column_names)


valores_nulos = {col: data[col].isnull().sum() for col in data.columns}
print("Número de valores nulos en cada columna:\n", valores_nulos)


Número de datos únicos en 'ESTU_PRGM_ACADEMICO': 948
Conteo de valores en 'ESTU_PRGM_ACADEMICO':
 {'DERECHO': np.int64(53244), 'ADMINISTRACION DE EMPRESAS': np.int64(51902), 'CONTADURIA PUBLICA': np.int64(39664), 'PSICOLOGIA': np.int64(31932), 'INGENIERIA INDUSTRIAL': np.int64(28481)}
Agrupación por 'ESTU_PRGM_ACADEMICO':
 {'3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES': np.int64(69), 'ACTIVIDAD FISICA Y DEPORTE': np.int64(162), 'ACUICULTURA': np.int64(79), 'ADMINISTRACION': np.int64(117), 'ADMINISTRACION  FINANCIERA': np.int64(74)}
Nombres de las columnas:
 ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4']
Númer

# **Preprocesamiento de datos**

In [6]:

columnas_redundantes = ['FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD']
data.drop(columnas_redundantes, axis=1, inplace=True)



In [7]:
print(data['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64


In [8]:

mapeo_rangos = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].map(mapeo_rangos)

print(data['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
3    175619
0    172987
1    172275
2    171619
Name: count, dtype: int64


In [9]:



columnas_booleanas = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']


for columna in columnas_booleanas:
    valores_unicos = set(data[columna])
    print(f"Valores únicos en la columna '{columna}': {valores_unicos}")


conversion_dict = {'Si': True, 'No': False}

for columna in columnas_booleanas:
    data[columna] = data[columna].replace(conversion_dict)


Valores únicos en la columna 'FAMI_TIENEINTERNET': {'Si', nan, 'No'}
Valores únicos en la columna 'FAMI_TIENELAVADORA': {'Si', nan, 'No'}
Valores únicos en la columna 'FAMI_TIENEAUTOMOVIL': {'Si', nan, 'No'}
Valores únicos en la columna 'ESTU_PAGOMATRICULAPROPIO': {'Si', nan, 'No'}
Valores únicos en la columna 'FAMI_TIENECOMPUTADOR': {'Si', nan, 'No'}


In [10]:

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato', '').str.strip()
data['FAMI_ESTRATOVIVIENDA'].replace('Sin', '0', inplace=True)
data['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(data['FAMI_ESTRATOVIVIENDA'])

print(data['FAMI_ESTRATOVIVIENDA'].head(10))


/tmp/ipython-input-10-47341938.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['FAMI_ESTRATOVIVIENDA'].replace('Sin', '0', inplace=True)


0    3.0
1    3.0
2    3.0
3    4.0
4    3.0
5    5.0
6    2.0
7    2.0
8    1.0
9    5.0
Name: FAMI_ESTRATOVIVIENDA, dtype: float64


In [11]:

data = pd.get_dummies(data, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='Valor_Matricula_Universidad')

data = pd.get_dummies(data, columns=['ESTU_HORASSEMANATRABAJA'], prefix='Horas_Trabajo')


print(data)

            ID  PERIODO                  ESTU_PRGM_ACADEMICO  \
0       904256    20212                           ENFERMERIA   
1       645256    20212                              DERECHO   
2       308367    20203                MERCADEO Y PUBLICIDAD   
3       470353    20195           ADMINISTRACION DE EMPRESAS   
4       989032    20212                           PSICOLOGIA   
...        ...      ...                                  ...   
692495   25096    20195                             BIOLOGIA   
692496  754213    20212                           PSICOLOGIA   
692497  504185    20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
692498  986620    20195                           PSICOLOGIA   
692499  933374    20195                           PSICOLOGIA   

       ESTU_PRGM_DEPARTAMENTO  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET  \
0                      BOGOTÁ                   3.0               True   
1                   ATLANTICO                   3.0              False   
2        

In [12]:
for columna in data.columns:

    conteo_valores = data[columna].value_counts()

    print(f"Valores de la columna '{columna}':")
    print(conteo_valores)
    print()

Valores de la columna 'ID':
ID
933374    1
904256    1
645256    1
308367    1
706675    1
         ..
257869    1
11829     1
47159     1
659872    1
989032    1
Name: count, Length: 692500, dtype: int64

Valores de la columna 'PERIODO':
PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

Valores de la columna 'ESTU_PRGM_ACADEMICO':
ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
DEPORTE Y ACTIVIDADA FISICA                                   1
QUIMICA Y FARMACIA                                            1
LICEN

In [13]:

for columna in data.columns:

    print(f"Valores únicos en la columna '{columna}':")

    print(data[columna].unique())
    print()


Valores únicos en la columna 'ID':
[904256 645256 308367 ... 504185 986620 933374]

Valores únicos en la columna 'PERIODO':
[20212 20203 20195 20183 20194 20213 20184 20202 20196]

Valores únicos en la columna 'ESTU_PRGM_ACADEMICO':
['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACIÓN EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACIÓN FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MÚSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL' 'INGENIERIA CIVIL'
 'CONTADURIA PÚBLICA' 'ADMINISTRACION EN SALUD'
 'ADMINISTRACIÓN DE EMPRESAS' 'ESTADISTICA' 'LICENCIATURA EN BIOLOGIA'
 'INGENIERIA AGROINDUSTRIAL' 'ZOOTECNIA' 'COMUNICACION AUDIOVISUAL'
 'LICENCIATURA EN EDU

In [14]:

for columna in data.columns:
    suma_nulos = data[columna].isna().sum()
    print(f"Columna '{columna}': {suma_nulos} valores nulos")

Columna 'ID': 0 valores nulos
Columna 'PERIODO': 0 valores nulos
Columna 'ESTU_PRGM_ACADEMICO': 0 valores nulos
Columna 'ESTU_PRGM_DEPARTAMENTO': 0 valores nulos
Columna 'FAMI_ESTRATOVIVIENDA': 32137 valores nulos
Columna 'FAMI_TIENEINTERNET': 26629 valores nulos
Columna 'FAMI_EDUCACIONPADRE': 23178 valores nulos
Columna 'FAMI_TIENELAVADORA': 39773 valores nulos
Columna 'FAMI_TIENEAUTOMOVIL': 43623 valores nulos
Columna 'ESTU_PAGOMATRICULAPROPIO': 6498 valores nulos
Columna 'FAMI_TIENECOMPUTADOR': 38103 valores nulos
Columna 'FAMI_EDUCACIONMADRE': 23664 valores nulos
Columna 'RENDIMIENTO_GLOBAL': 0 valores nulos
Columna 'coef_1': 0 valores nulos
Columna 'coef_2': 0 valores nulos
Columna 'coef_3': 0 valores nulos
Columna 'coef_4': 0 valores nulos
Columna 'Valor_Matricula_Universidad_Entre 1 millón y menos de 2.5 millones': 0 valores nulos
Columna 'Valor_Matricula_Universidad_Entre 2.5 millones y menos de 4 millones': 0 valores nulos
Columna 'Valor_Matricula_Universidad_Entre 4 millones 

In [15]:
print(data.isnull().sum())

ID                                                                          0
PERIODO                                                                     0
ESTU_PRGM_ACADEMICO                                                         0
ESTU_PRGM_DEPARTAMENTO                                                      0
FAMI_ESTRATOVIVIENDA                                                    32137
FAMI_TIENEINTERNET                                                      26629
FAMI_EDUCACIONPADRE                                                     23178
FAMI_TIENELAVADORA                                                      39773
FAMI_TIENEAUTOMOVIL                                                     43623
ESTU_PAGOMATRICULAPROPIO                                                 6498
FAMI_TIENECOMPUTADOR                                                    38103
FAMI_EDUCACIONMADRE                                                     23664
RENDIMIENTO_GLOBAL                                              

In [16]:



moda = data.mode().iloc[0]
data.fillna(moda, inplace=True)


print("\nDatos  de la imputación de valores nulos:")
print(data.head())


print("\nModa utilizada para la imputación:")
print(moda)

/tmp/ipython-input-16-436110863.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(moda, inplace=True)



Datos  de la imputación de valores nulos:
       ID  PERIODO         ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0  904256    20212                  ENFERMERIA                 BOGOTÁ   
1  645256    20212                     DERECHO              ATLANTICO   
2  308367    20203       MERCADEO Y PUBLICIDAD                 BOGOTÁ   
3  470353    20195  ADMINISTRACION DE EMPRESAS              SANTANDER   
4  989032    20212                  PSICOLOGIA              ANTIOQUIA   

   FAMI_ESTRATOVIVIENDA  FAMI_TIENEINTERNET  \
0                   3.0                True   
1                   3.0               False   
2                   3.0                True   
3                   4.0                True   
4                   3.0                True   

                  FAMI_EDUCACIONPADRE  FAMI_TIENELAVADORA  \
0    Técnica o tecnológica incompleta                True   
1      Técnica o tecnológica completa                True   
2  Secundaria (Bachillerato) completa               

# **Entrenar el modelo**

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split


X = data.drop("RENDIMIENTO_GLOBAL", axis=1)
y = data["RENDIMIENTO_GLOBAL"]


X = pd.get_dummies(X)


X, y = X.sample(n=1000, random_state=42), y.sample(n=1000, random_state=42)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf = RandomForestClassifier(random_state=42)
scores = cross_val_score(rf, X_train, y_train, cv=3)
print("Cross-validation scores:", scores)
print("Mean CV score:", scores.mean())


rf.fit(X_train, y_train)


accuracy = rf.score(X_test, y_test)
print("Accuracy on test set:", accuracy)


Cross-validation scores: [0.34456929 0.3258427  0.37218045]
Mean CV score: 0.3475308120488487
Accuracy on test set: 0.285


# **Preprocesamiento y predicción del conjunto Test**

In [18]:
test_data = pd.read_csv("datos/test.csv")


In [19]:


datos_unicos = len(set(data['ESTU_PRGM_ACADEMICO']))
print("Número de datos únicos en 'ESTU_PRGM_ACADEMICO':", datos_unicos)


educacion_madre_counts = dict(data['ESTU_PRGM_ACADEMICO'].value_counts().head())
print("Conteo de valores en 'ESTU_PRGM_ACADEMICO':\n", educacion_madre_counts)


conteo_valores = dict(data['ESTU_PRGM_ACADEMICO'].groupby(data['ESTU_PRGM_ACADEMICO']).size().head())
print("Agrupación por 'ESTU_PRGM_ACADEMICO':\n", conteo_valores)


column_names = [col for col in data.columns]
print("Nombres de las columnas:\n", column_names)


valores_nulos = {col: data[col].isnull().sum() for col in data.columns}
print("Número de valores nulos en cada columna:\n", valores_nulos)

Número de datos únicos en 'ESTU_PRGM_ACADEMICO': 948
Conteo de valores en 'ESTU_PRGM_ACADEMICO':
 {'DERECHO': np.int64(53244), 'ADMINISTRACION DE EMPRESAS': np.int64(51902), 'CONTADURIA PUBLICA': np.int64(39664), 'PSICOLOGIA': np.int64(31932), 'INGENIERIA INDUSTRIAL': np.int64(28481)}
Agrupación por 'ESTU_PRGM_ACADEMICO':
 {'3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES': np.int64(69), 'ACTIVIDAD FISICA Y DEPORTE': np.int64(162), 'ACUICULTURA': np.int64(79), 'ADMINISTRACION': np.int64(117), 'ADMINISTRACION  FINANCIERA': np.int64(74)}
Nombres de las columnas:
 ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'Valor_Matricula_Universidad_Entre 1 millón y menos de 2.5 millones', 'Valor_Matricula_Universidad_Entre 2.5 millon

In [20]:

test_data = test_data.drop(['FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD'], axis=1)


In [21]:

columnas_booleanas = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']


for columna in columnas_booleanas:
    valores_unicos = set(data[columna])
    print(f"Valores únicos en la columna '{columna}': {valores_unicos}")


conversion_dict = {'Si': True, 'No': False}

for columna in columnas_booleanas:
    test_data[columna] = test_data[columna].replace(conversion_dict)


Valores únicos en la columna 'FAMI_TIENEINTERNET': {False, True}
Valores únicos en la columna 'FAMI_TIENELAVADORA': {False, True}
Valores únicos en la columna 'FAMI_TIENEAUTOMOVIL': {False, True}
Valores únicos en la columna 'ESTU_PAGOMATRICULAPROPIO': {False, True}
Valores únicos en la columna 'FAMI_TIENECOMPUTADOR': {False, True}


In [22]:
test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato', '').str.strip()
test_data['FAMI_ESTRATOVIVIENDA'].replace('Sin', '0', inplace=True)
test_data['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(test_data['FAMI_ESTRATOVIVIENDA'])

print(test_data['FAMI_ESTRATOVIVIENDA'].head(10))

0    3.0
1    2.0
2    3.0
3    1.0
4    2.0
5    1.0
6    3.0
7    4.0
8    2.0
9    2.0
Name: FAMI_ESTRATOVIVIENDA, dtype: float64


/tmp/ipython-input-22-604752262.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['FAMI_ESTRATOVIVIENDA'].replace('Sin', '0', inplace=True)


In [23]:


print("Columnas en el DataFrame:")
print(data.columns)


if 'ESTU_VALORMATRICULAUNIVERSIDAD' in data.columns and 'ESTU_HORASSEMANATRABAJA' in data.columns:

    test_data = pd.get_dummies(data, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='Valor_Matricula_Universidad')


    test_data = pd.get_dummies(test_data, columns=['ESTU_HORASSEMANATRABAJA'], prefix='Horas_Trabajo')

    print("Datos después de la codificación One-Hot:")
    print(test_data.head())



Columnas en el DataFrame:
Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'coef_1', 'coef_2', 'coef_3', 'coef_4',
       'Valor_Matricula_Universidad_Entre 1 millón y menos de 2.5 millones',
       'Valor_Matricula_Universidad_Entre 2.5 millones y menos de 4 millones',
       'Valor_Matricula_Universidad_Entre 4 millones y menos de 5.5 millones',
       'Valor_Matricula_Universidad_Entre 5.5 millones y menos de 7 millones',
       'Valor_Matricula_Universidad_Entre 500 mil y menos de 1 millón',
       'Valor_Matricula_Universidad_Menos de 500 mil',
       'Valor_Matricula_Universidad_Más de 7 millones',
       'Valor_Matricula_Universidad_No pagó matrícula', 'Horas_Trabajo_0',
       'Horas_Trabajo_Entre 11 y 20 horas',
      

In [24]:
for columna in data.columns:

    conteo_valores = data[columna].value_counts()

    print(f"Valores de la columna '{columna}':")
    print(conteo_valores)
    print()

Valores de la columna 'ID':
ID
933374    1
904256    1
645256    1
308367    1
706675    1
         ..
257869    1
11829     1
47159     1
659872    1
989032    1
Name: count, Length: 692500, dtype: int64

Valores de la columna 'PERIODO':
PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

Valores de la columna 'ESTU_PRGM_ACADEMICO':
ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
DEPORTE Y ACTIVIDADA FISICA                                   1
QUIMICA Y FARMACIA                                            1
LICEN

In [25]:

for columna in data.columns:

    print(f"Valores únicos en la columna '{columna}':")

    print(data[columna].unique())
    print()


Valores únicos en la columna 'ID':
[904256 645256 308367 ... 504185 986620 933374]

Valores únicos en la columna 'PERIODO':
[20212 20203 20195 20183 20194 20213 20184 20202 20196]

Valores únicos en la columna 'ESTU_PRGM_ACADEMICO':
['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACIÓN EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACIÓN FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MÚSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL' 'INGENIERIA CIVIL'
 'CONTADURIA PÚBLICA' 'ADMINISTRACION EN SALUD'
 'ADMINISTRACIÓN DE EMPRESAS' 'ESTADISTICA' 'LICENCIATURA EN BIOLOGIA'
 'INGENIERIA AGROINDUSTRIAL' 'ZOOTECNIA' 'COMUNICACION AUDIOVISUAL'
 'LICENCIATURA EN EDU

In [26]:
print(test_data.isnull().sum())

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     2723
ESTU_HORASSEMANATRABAJA           13379
FAMI_ESTRATOVIVIENDA              13795
FAMI_TIENEINTERNET                11539
FAMI_EDUCACIONPADRE                9993
FAMI_TIENELAVADORA                17259
FAMI_TIENEAUTOMOVIL               18918
ESTU_PAGOMATRICULAPROPIO           2807
FAMI_TIENECOMPUTADOR              16439
FAMI_EDUCACIONMADRE               10223
coef_1                                0
coef_2                                0
coef_3                                0
coef_4                                0
dtype: int64


In [27]:

moda = data.mode().iloc[0]
data.fillna(moda, inplace=True)


print("\nDatos  de la imputación de valores nulos:")
print(test_data.head())


print("\nModa utilizada para la imputación:")
print(moda)


Datos  de la imputación de valores nulos:
       ID  PERIODO                     ESTU_PRGM_ACADEMICO  \
0  550236    20183                          TRABAJO SOCIAL   
1   98545    20203  ADMINISTRACION COMERCIAL Y DE MERCADEO   
2  499179    20212                  INGENIERIA MECATRONICA   
3  782980    20195                      CONTADURIA PUBLICA   
4  785185    20212              ADMINISTRACION DE EMPRESAS   

  ESTU_PRGM_DEPARTAMENTO            ESTU_VALORMATRICULAUNIVERSIDAD  \
0                BOLIVAR                          Menos de 500 mil   
1              ANTIOQUIA  Entre 2.5 millones y menos de 4 millones   
2                 BOGOTÁ    Entre 1 millón y menos de 2.5 millones   
3                  SUCRE    Entre 1 millón y menos de 2.5 millones   
4              ATLANTICO  Entre 2.5 millones y menos de 4 millones   

  ESTU_HORASSEMANATRABAJA  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET  \
0       Menos de 10 horas                   3.0               True   
1     Entre 21 y 30 hor

In [28]:

test_data = pd.get_dummies(test_data)


missing_cols = set(X_train.columns) - set(test_data.columns)
for c in missing_cols:
    test_data[c] = 0
test_data = test_data[X_train.columns]

In [29]:

import pandas as pd

test_data = pd.read_csv("datos/test.csv")
train_data = pd.read_csv("datos/train.csv")


print(train_data.head())


print(test_data.head())


       ID  PERIODO         ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0  904256    20212                  ENFERMERIA                 BOGOTÁ   
1  645256    20212                     DERECHO              ATLANTICO   
2  308367    20203       MERCADEO Y PUBLICIDAD                 BOGOTÁ   
3  470353    20195  ADMINISTRACION DE EMPRESAS              SANTANDER   
4  989032    20212                  PSICOLOGIA              ANTIOQUIA   

             ESTU_VALORMATRICULAUNIVERSIDAD ESTU_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones       Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                       0   
2  Entre 2.5 millones y menos de 4 millones         Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                       0   
4  Entre 2.5 millones y menos de 4 millones     Entre 21 y 30 horas   

  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET                 FAMI_EDUCACIONPADRE  \
0            Estrato 3                 Si    Técnica 

In [30]:

moda = train_data.mode().iloc[0]


test_data.fillna(moda, inplace=True)


print("\nDatos de la imputación de valores nulos en el conjunto de prueba:")
print(test_data.head())



Datos de la imputación de valores nulos en el conjunto de prueba:
       ID  PERIODO                     ESTU_PRGM_ACADEMICO  \
0  550236    20183                          TRABAJO SOCIAL   
1   98545    20203  ADMINISTRACION COMERCIAL Y DE MERCADEO   
2  499179    20212                  INGENIERIA MECATRONICA   
3  782980    20195                      CONTADURIA PUBLICA   
4  785185    20212              ADMINISTRACION DE EMPRESAS   

  ESTU_PRGM_DEPARTAMENTO            ESTU_VALORMATRICULAUNIVERSIDAD  \
0                BOLIVAR                          Menos de 500 mil   
1              ANTIOQUIA  Entre 2.5 millones y menos de 4 millones   
2                 BOGOTÁ    Entre 1 millón y menos de 2.5 millones   
3                  SUCRE    Entre 1 millón y menos de 2.5 millones   
4              ATLANTICO  Entre 2.5 millones y menos de 4 millones   

  ESTU_HORASSEMANATRABAJA FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET  \
0       Menos de 10 horas            Estrato 3                 Si   
1

In [31]:

test_data = pd.get_dummies(test_data)


missing_cols = set(X_train.columns) - set(test_data.columns)
for c in missing_cols:
    test_data[c] = 0
test_data = test_data[X_train.columns]


print("\nInformación de las columnas en el conjunto de prueba:")
print(test_data.info())


print("\nPrimera fila del conjunto de prueba después del preprocesamiento:")
print(test_data.head(1))


test_data.to_csv('preprocessed_test.csv', index=False)




Información de las columnas en el conjunto de prueba:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Columns: 1028 entries, ID to FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
dtypes: bool(964), float64(4), int64(60)
memory usage: 417.8 MB
None

Primera fila del conjunto de prueba después del preprocesamiento:
       ID  PERIODO  FAMI_ESTRATOVIVIENDA  FAMI_TIENEINTERNET  \
0  550236    20183                     0                   0   

   FAMI_TIENELAVADORA  FAMI_TIENEAUTOMOVIL  ESTU_PAGOMATRICULAPROPIO  \
0                   0                    0                         0   

   FAMI_TIENECOMPUTADOR  coef_1  coef_2  ...  FAMI_EDUCACIONMADRE_Ninguno  \
0                     0   0.328   0.219  ...                        False   

   FAMI_EDUCACIONMADRE_No Aplica  FAMI_EDUCACIONMADRE_No sabe  \
0                          False                        False   

   FAMI_EDUCACIONMADRE_Postgrado  FAMI_EDUCACIONMADRE_Primaria completa  \
0             

In [32]:

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', '0')


data['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(data['FAMI_ESTRATOVIVIENDA'], errors='coerce')


data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].astype(int)


data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].astype(str).str.replace('Estrato ', '')


data = pd.get_dummies(data)


X = data.drop('RENDIMIENTO_GLOBAL', axis=1)
y = data['RENDIMIENTO_GLOBAL']


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


X_val = pd.get_dummies(X_val)


missing_cols = set(X_train.columns) - set(X_val.columns)
for c in missing_cols:
    X_val[c] = 0
X_val = X_val[X_train.columns]


print("\nInformación de las columnas en el conjunto de validación:")
print(X_val.info())


print("\nPrimera fila del conjunto de validación después del preprocesamiento:")
print(X_val.head(1))


X_val.to_csv('preprocessed_validation.csv', index=False)



Información de las columnas en el conjunto de validación:
<class 'pandas.core.frame.DataFrame'>
Index: 138500 entries, 137278 to 628411
Columns: 1034 entries, ID to FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
dtypes: bool(1028), float64(4), int64(2)
memory usage: 143.2 MB
None

Primera fila del conjunto de validación después del preprocesamiento:
            ID  PERIODO  FAMI_TIENEINTERNET  FAMI_TIENELAVADORA  \
137278  345759    20195                True                True   

        FAMI_TIENEAUTOMOVIL  ESTU_PAGOMATRICULAPROPIO  FAMI_TIENECOMPUTADOR  \
137278                 True                     False                  True   

        coef_1  coef_2  coef_3  ...  FAMI_EDUCACIONMADRE_Ninguno  \
137278    0.24   0.303   0.276  ...                        False   

        FAMI_EDUCACIONMADRE_No Aplica  FAMI_EDUCACIONMADRE_No sabe  \
137278                          False                        False   

        FAMI_EDUCACIONMADRE_Postgrado  FAMI_EDUCACIONMADRE_Primaria c

In [33]:


import pandas as pd

data = pd.read_csv("datos/train.csv", nrows=10000)  # Leer solo las primeras 10,000 filas


X = data.drop(["RENDIMIENTO_GLOBAL", "ID"], axis=1)
y = data["RENDIMIENTO_GLOBAL"]
ids = data["ID"]


X = pd.get_dummies(X)


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(X, y, ids, test_size=0.2, random_state=42)



from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_val)


from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy en el conjunto de validación: {accuracy:.2f}')


for id, pred in zip(ids_val, y_pred):
    print(f"ID: {id}, Predicción: {pred}")

Accuracy en el conjunto de validación: 0.39
ID: 656009, Predicción: alto
ID: 498978, Predicción: medio-alto
ID: 28672, Predicción: medio-bajo
ID: 988749, Predicción: medio-alto
ID: 972171, Predicción: medio-bajo
ID: 877691, Predicción: medio-bajo
ID: 364325, Predicción: medio-bajo
ID: 352257, Predicción: bajo
ID: 76851, Predicción: alto
ID: 951818, Predicción: medio-bajo
ID: 33086, Predicción: bajo
ID: 740154, Predicción: medio-alto
ID: 603595, Predicción: bajo
ID: 105123, Predicción: medio-bajo
ID: 712951, Predicción: bajo
ID: 223428, Predicción: medio-alto
ID: 197805, Predicción: medio-bajo
ID: 218290, Predicción: bajo
ID: 626891, Predicción: medio-bajo
ID: 34709, Predicción: bajo
ID: 197147, Predicción: alto
ID: 293946, Predicción: alto
ID: 481044, Predicción: bajo
ID: 25896, Predicción: bajo
ID: 90114, Predicción: bajo
ID: 301202, Predicción: alto
ID: 2655, Predicción: medio-alto
ID: 194519, Predicción: alto
ID: 832025, Predicción: medio-alto
ID: 510045, Predicción: medio-alto
ID: 

In [34]:

test_data = pd.read_csv("datos/test.csv")


test_ids = test_data['ID']

In [35]:

test_data = pd.get_dummies(test_data)

missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

In [37]:

test_predictions = model.predict(test_data)


submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': test_predictions})
submission.to_csv('submission2.csv', index=False)
print("Predicciones guardadas submission2.csv")

Predicciones guardadas submission2.csv
